In [1]:
# https://www.zhihu.com/question/54152397
# 

### 推送消息


In [ ]:
from rocketmq.client import Producer, Message
producer = Producer('PID-001')     #随便
producer.set_namesrv_addr('ip:port')   #ip和端口
producer.start()
 
msg = Message('rocket_mq_test_broadcast_topic')  #topic
msg.set_keys('2020-12-15')
msg.set_tags('explain')
msg.set_body('{"key":"value"}')
ret = producer.send_sync(msg)
print(ret.status, ret.msg_id, ret.offset)
producer.shutdown()

In [ ]:
# 推送消息的时候，如果消息所占字节太长，需要手动设置size，代码中设置的是1M。
producer = Producer('PID-001',max_message_size=1024*1024)

In [ ]:
import json
from rocketmq.client import Producer, Message

producer = Producer('PID-XXX')
producer.set_namesrv_domain('http://onsaddr-internet.aliyun.com/rocketmq/nsaddr4client-internet')#rocketmq队列接口地址（服务器ip:port）
# For ip and port name server address, use `set_namesrv_addr` method, for example:
# producer.set_namesrv_addr('127.0.0.1:9887')
producer.set_session_credentials('XXX', 'XXXX', 'ALIYUN')#可以不使用
producer.start()

msg_body = {"id": "test_id", "name": "test_name", "message": "test_message"}
ss = json.dumps(msg_body).encode('utf-8')

msg = Message('YOUR-TOPIC') #topic名称
msg.set_keys('XXX')#每个消息在业务层面的唯一标识码，要设置到keys字段，方便将来定位消息丢失问题。服务器会为每个消息创建索引（哈希索引），应用可以通过topic，key来查询这条消息内容，以及消息被谁消费。由于是哈希索引，请务必保证key尽可能唯一，这样可以避免潜在的哈希冲突。
msg.set_tags('XXX')#一个应用尽可能用一个Topic，消息子类型用tags来标识，tags可以由应用自由设置。只有发送消息设置了tags，消费方在订阅消息时，才可以利用tags在broker做消息过滤。
msg.set_body(ss)
ret = producer.send_sync(msg)
print(ret.status, ret.msg_id, ret.offset)
producer.shutdown()


### 消费方式PullConsumer（全部消费）（可重复消费）

In [ ]:
from rocketmq.client import PullConsumer
import json
consumer = PullConsumer('PID-001')
consumer.set_namesrv_addr('ip:port')
consumer.start()
for msg in consumer.pull('rocket_mq_test_broadcast_topic'):
    print(msg.tags)
    print(msg.keys)
    print(msg.id, msg.body)
    print(msg.topic)
    print(msg)
    data=json.loads(str(msg))  #dict
consumer.shutdown()

In [ ]:
from rocketmq.client import PullConsumer

consumer = PullConsumer('CID_XXX')
consumer.set_namesrv_domain('http://onsaddr-internet.aliyun.com/rocketmq/nsaddr4client-internet')
# For ip and port name server address, use `set_namesrv_addr` method, for example:
# consumer.set_namesrv_addr('127.0.0.1:9887')

consumer.set_session_credentials('XXX', 'XXXX', 'ALIYUN')
consumer.start()

for msg in consumer.pull('YOUR-TOPIC'):
    print(msg.id, msg.body)

consumer.shutdown()

### 消费方式PushConsumer（即时消费）（不可重复消费）

In [ ]:
import time
from rocketmq.client import PushConsumer
def callback(msg):
    print(msg)
consumer = PushConsumer('PID-001')
consumer.set_namesrv_addr('ip:port')
consumer.subscribe("rocket_mq_test_broadcast_topic", callback)
consumer.start()
while True:
    time.sleep(30)
consumer.shutdown()

In [ ]:
import time

from rocketmq.client import PushConsumer

def callback(msg):
    print(msg.id, msg.body)

consumer = PushConsumer('CID_XXX')
consumer.set_namesrv_domain('http://onsaddr-internet.aliyun.com/rocketmq/nsaddr4client-internet')
# For ip and port name server address, use `set_namesrv_addr` method, for example:
# consumer.set_namesrv_addr('127.0.0.1:9887')

consumer.set_session_credentials('XXX', 'XXXX', 'ALIYUN')
consumer.subscribe('YOUR-TOPIC', callback)
consumer.start()

while True:
    time.sleep(3600)

consumer.shutdown()


### 控制日志的输出频率

In [ ]:
from rocketmq.client import dll

dll.SetPushConsumerLogLevel(namesrv_addr.encode('utf-8'), 1)